In [1]:
import sys
import copy
from datetime import datetime

import torch
import torchvision
import torch.nn as nn
import torchvision.transforms as transforms
from torch.optim.optimizer import Optimizer, required

from logger import Logger

from solver import Solver
from torch.nn import functional as F
from crossngover import CrossN

torch.manual_seed(666)

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=512,
                                          shuffle=False, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=512,
                                         shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
from sklearn.metrics import f1_score

In [4]:
def train_models(params, net, device=0):
    
    mode = params['mode']
    evo_step = int(params['evo_step'])
    
    experiment_note = ''
    path = ''
    for key in params:
        experiment_note += key +'_'+ params[key]+'\n'
        path +=  '_'+ params[key]
    

    logger = Logger(path, experiment_note)
    
    print(logger.path)
    print(experiment_note)

    lr = 0.001

    optimizer = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    evo_optim = CrossN()

    def validation(net, dataloader, loss_fn=None, device=0):
        correct = 0
        total = 0
        total_loss = 0
        counter = 0
        with torch.no_grad():
            for data in dataloader:
                images, labels = data
                images = images.cuda(device)
                labels = labels.cuda(device)
                outputs = net(images)
                if loss_fn:
                    total_loss += loss_fn(outputs, labels)
                _, predicted = torch.max(outputs.data, 1)
                total += f1_score(predicted.cpu().numpy(), labels.cpu().numpy(),average="micro")
                counter += 1
        if loss_fn:
            return -1 * total_loss / counter, 100.0 * (total/counter)
        else:
            return 100.0 * (total/counter)

    
    solver = Solver(
        net,
        optimizer,
        logger,
        criterion,
        validation,
        evo_optim, 
        trainloader,
        testloader,
        testloader,  
        epochs=100,
        evo_step=evo_step,
        child_count=20,
        best_child_count=3,
        mode=mode,
        debug=True,
        lr=lr,
        device=device)

    logger.add_post_result(f'start: {datetime.now()}')
    solver.start()
    logger.add_post_result(f'finish: {datetime.now()}')
    torch.save(net.state_dict(), logger.path + '/model_last.chk')
    logger.close()
    
    

def train_three_types(model, TF, name):
    modes = ['evo_only', 'gradient', 'evo_cross']
    
    evo_step = 10

    for mode in modes: 
        orig_stdout = sys.stdout
        f = open(f'outputs/{name}_{mode}.txt', 'w')
        sys.stdout = f

        
        params = {'net_name':name,
             'preptrained':TF,
             'mode':mode,
             'evo_step':str(evo_step)}
        temp_model = copy.deepcopy(model)
        train_models(params, temp_model)
        sys.stdout = orig_stdout
        f.close()
        torch.cuda.empty_cache()
    print('Finished')

In [5]:
def init_weights(m):
    if ((type(m) == nn.Linear) | (type(m) == nn.Conv2d)):
        torch.nn.init.uniform_(m.weight.data)
        m.bias.data.fill_(0.00)

In [6]:
net = torchvision.models.resnet18(pretrained=False)

num_ftrs = net.fc.in_features
classes = ('plane', 'car', 'bird', 'cat',
        'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
net.classifier = nn.Linear(num_ftrs, len(classes))
init_weights(net)
net.cuda()
print()

In [7]:
train_three_types(net, 'F', 'resnet18_256_test_lr')

KeyboardInterrupt: 

mobilenet from F:
- Evo + Cross - 
- Evo Only - 
- Standard - 

resnet from F:
- Evo + Cross  - 
- Evo Only - 
- Standard - 

mobilenet pretrained T:
- Evo + Cross - 
- Evo Only  - 
- Standard - 

resnet pretrained T:
- Evo + Cross  
- Evo Only 
- Standard 